In [ ]:
# Run me first
!apt update
!pip install pydrive2

In [ ]:
# Run me after the first code
from pydrive2 import auth, drive, files
from google.colab import auth as colabauth
from oauth2client.client import GoogleCredentials
import os
import shutil
import requests
import json
import argparse

_goindex_url = "PUT_GOINDEX_URL_HERE"  # for example "	https://animebatchddl.vi-filter-kun.workers.dev/0:/"
_gdfolderid = "PUT_GD_FOLDER_ID_HERE"  # the folder on google drive where all files should be uploaded
http_auth = ("USERNAME", "PASSWORD")  # Use this to authentificate via http auth
goindex_password = ""  # download auth password
stop_download_after = 725000000000  # 720 GB
traffic_counter = 0

# Google Auth https://colab.research.google.com/notebooks/io.ipynb
colabauth.authenticate_user()
gauth = auth.GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = drive.GoogleDrive(gauth)
with open("client_secrets.json", "w", encoding="utf-8") as writer:#shutil.copy("adc.json", "client_secrets.json")
  writer.write("""{"web": {"client_id": "32555940559.apps.googleusercontent.com", "project_id": "powershell", "auth_uri": "https://accounts.google.com/o/oauth2/auth", "token_uri": "https://oauth2.googleapis.com/token", "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs", "client_secret": "ZmssLNjJy2998hD4CTg2ejr2", "redirect_uris": ["urn:ietf:wg:oauth:2.0:oob", "http://localhost"]}}""")

# Fix URL
if not _goindex_url.endswith("/"):
  _goindex_url += "/"

def _get_data(url):
  # Gets files and folders from goindex website
  f = []
  data = {
      "page_index": 0,
      "page_token": None,
      "password": goindex_password,
      "q": ""
  }
  request = {"nextPageToken": "yes"}  # Dummy
  while "nextPageToken" in request and request["nextPageToken"]:
    request = requests.post(url, data=json.dumps(data), auth=http_auth).json()
    f = f + request["data"]["files"]
    data["page_index"] += 1
    data["page_token"] = request["nextPageToken"]
  return f

def _get_gd_files(gid):
  # Gets files and folders from Google Drive
  data = {}
  files = drive.ListFile({'q': "'{}' in parents and trashed=false".format(gid)}).GetList()
  for file in files:
    data[file["title"]] = {"id": file["id"], "mimeType": file["mimeType"]}
  return data

def dir(url, foldername, gdfolderid, depth=1):
  global traffic_counter
  print(f'{depth*"-"}>{foldername}')
  
  gd_files = _get_gd_files(gdfolderid)  # Get GD files & folders
  files = _get_data(url)  # Get GoIndex files & folders
  for file in files:  # Cycle through files & folders
    if file["mimeType"] == "application/vnd.google-apps.folder":  # If folder
      if file["name"] in gd_files:  # if folder already exist take id
        sub_gdfolderid = gd_files[file["name"]]["id"]
      else:  # if not exist create folder and take id
        new_folder = drive.CreateFile({'title': file["name"], 'mimeType': 'application/vnd.google-apps.folder', "parents": [{"id": gdfolderid}]})
        new_folder.Upload()
        sub_gdfolderid = new_folder["id"]
      dir(url=url+f"{file['name']}/", foldername=file["name"], gdfolderid=sub_gdfolderid, depth=depth+1)  # go through subfolder
    else:  # Else file
      if file["name"] not in gd_files:  # check if file not in folder, if exist skip
        traffic_counter += int(file["size"])  # update traffic counter
        if traffic_counter > stop_download_after:  # if traffic limit reached, disconnect abruptly
          os._exit(1)
        print(f'{depth*"-"}↴{file["name"]} [{int(int(file["size"])/1024/1024)}MB]')  # print downloading file
        down_and_up(url+file['name'], file['name'], gdfolderid)  # download and upload file

def down_and_up(url, filename, gid):
  # Stolen from the web
  with requests.get(url, stream=True, auth=http_auth) as r:  # download file
      r.raise_for_status()
      with open(filename, 'wb') as f:
          for chunk in r.iter_content(chunk_size=8192*1000): 
              f.write(chunk)
  file = drive.CreateFile({"title": filename, "parents": [{"id": gid}]})  # Upload to Google drive
  file.SetContentFile(filename)
  file.Upload()
  os.remove(filename)  # remove file

dir(url=_goindex_url, foldername="/", gdfolderid=_gdfolderid)

->/
-->[Moozzi2] 3-Choume no Tama Uchi no Tama Shirimasenka - TV + SP
-->[Moozzi2] 3-gatsu no Lion - TV + SP
-->[Moozzi2] 3-gatsu no Lion S2 - TV + Tokuten DVD
--↴[Moozzi2] 3-gatsu no Lion S2 - 18 (BD 1920x1080 x.264 Flac).mkv [1541MB]
--↴[Moozzi2] 3-gatsu no Lion S2 - 19 (BD 1920x1080 x.264 Flac).mkv [1377MB]


KeyboardInterrupt: ignored